In [ ]:
!pip install isodate

Code zum Abrufen fast aller Daten des youtube Channels. Es müssen noch die API-Daten und der Link zur json-Datei eingetragen werden. Außerdem der Datumsbereich

In [ ]:
# API Konfiguration - Hier Ihre Werte eintragen
CHANNEL_ID = "UCSeil5V81-mEGB1-VNR7YEA"  # Ihre YouTube Channel ID
DATA_API_V3_KEY = ""  # API Key für YouTube Data API v3
#ANALYTICS_API_KEY = ""  # API Key für YouTube Analytics API
CREDENTIALS_PATH = ""


# Datum Konfiguration - Format: "DD.MM.YYYY"
START_DATE = "01.01.2024"  # z.B. "01.09.2024"
END_DATE = "31.01.2024"   # z.B. "02.09.2024"

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from datetime import datetime
import pandas as pd
import isodate

# OAuth 2.0 Scopes
SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

def get_video_data_from_api():
    """YouTube Data API v3 Abfrage für Basis-Videodaten"""
    youtube = build('youtube', 'v3', developerKey=DATA_API_V3_KEY)
    
    start_date = datetime.strptime(START_DATE, '%d.%m.%Y')
    end_date = datetime.strptime(END_DATE, '%d.%m.%Y').replace(hour=23, minute=59, second=59)
    
    start_date_rfc = start_date.isoformat() + 'Z'
    end_date_rfc = end_date.isoformat() + 'Z'
    
    videos_data = []
    next_page_token = None
    
    while True:
        request = youtube.search().list(
            part='snippet',
            channelId=CHANNEL_ID,
            maxResults=50,
            order='date',
            publishedAfter=start_date_rfc,
            publishedBefore=end_date_rfc,
            type='video',
            pageToken=next_page_token
        )
        
        response = request.execute()
        
        for item in response['items']:
            video_id = item['id']['videoId']
            
            video_request = youtube.videos().list(
                part='snippet,contentDetails,status',
                id=video_id
            )
            video_response = video_request.execute()
            
            if video_response['items']:
                video_info = video_response['items'][0]
                duration = int(isodate.parse_duration(video_info['contentDetails']['duration']).total_seconds())
                
                video_data = {
                    'video_id': video_id,
                    'title': video_info['snippet']['title'],
                    'publish_date': video_info['snippet']['publishedAt'],
                    'video_length_seconds': duration,
                    'privacy_status': video_info['status']['privacyStatus']
                }
                videos_data.append(video_data)
        
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    
    return pd.DataFrame(videos_data)

def get_analytics_data(analytics, video_id, start_date, end_date):
    """YouTube Analytics API Abfrage für detaillierte Metriken"""
    try:
        # Performance Metriken
        metrics = [
            'estimatedMinutesWatched',
            'averageViewDuration',
            'averageViewPercentage',
            'subscribersGained',
            'subscribersLost',
            'views',
            'likes',
            'dislikes',
            'shares',
            'comments'
        ]
        
        perf_response = analytics.reports().query(
            ids=f'channel=={CHANNEL_ID}',
            startDate=start_date.strftime('%Y-%m-%d'),
            endDate=end_date.strftime('%Y-%m-%d'),
            metrics=','.join(metrics),
            filters=f'video=={video_id}'
        ).execute()

        # Demografische Daten
        demo_response = analytics.reports().query(
            ids=f'channel=={CHANNEL_ID}',
            startDate=start_date.strftime('%Y-%m-%d'),
            endDate=end_date.strftime('%Y-%m-%d'),
            metrics='viewerPercentage',
            dimensions='ageGroup,gender',
            filters=f'video=={video_id}'
        ).execute()

        # Basis-Daten sammeln
        data = {
            'video_id': video_id,
            'wiedergabezeit_minuten': 0,
            'durchschnittliche_wiedergabedauer': 0,
            'durchschnittliche_wiedergabedauer_prozent': 0,
            'gewonnene_abonnenten': 0,
            'verlorene_abonnenten': 0,
            'aufrufe': 0,
            'likes': 0,
            'dislikes': 0,
            'geteilte_inhalte': 0,
            'kommentare': 0
        }

        # Performance Daten einfügen
        if 'rows' in perf_response and perf_response['rows']:
            row = perf_response['rows'][0]
            data.update({
                'wiedergabezeit_minuten': row[0],
                'durchschnittliche_wiedergabedauer': row[1],
                'durchschnittliche_wiedergabedauer_prozent': row[2],
                'gewonnene_abonnenten': row[3],
                'verlorene_abonnenten': row[4],
                'aufrufe': row[5],
                'likes': row[6],
                'dislikes': row[7],
                'geteilte_inhalte': row[8],
                'kommentare': row[9]
            })

        # Demografische Daten einfügen
        if 'rows' in demo_response:
            for row in demo_response['rows']:
                age_group, gender, percentage = row
                key = f"audience_{gender.lower()}_{age_group}"
                data[key] = percentage

        return data

    except Exception as e:
        print(f"Fehler bei Video {video_id}: {str(e)}")
        return None

def main():
    try:
        print("1. Hole Basis-Videodaten über YouTube Data API v3...")
        video_df = get_video_data_from_api()
        print(f"   - {len(video_df)} Videos gefunden")

        print("\n2. Hole Analytics-Daten über YouTube Analytics API...")
        # Authentifizierung für Analytics API
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
        credentials = flow.run_local_server(port=0)
        analytics = build('youtubeAnalytics', 'v2', credentials=credentials)
        
        start_date = datetime.strptime(START_DATE, '%d.%m.%Y')
        end_date = datetime.strptime(END_DATE, '%d.%m.%Y')
        
        # Sammle Analytics-Daten für jedes Video
        analytics_data = []
        for index, row in video_df.iterrows():
            print(f"   - Verarbeite Video {index + 1} von {len(video_df)}: {row['video_id']}")
            data = get_analytics_data(analytics, row['video_id'], start_date, end_date)
            if data:
                analytics_data.append(data)

        # Erstelle DataFrame aus Analytics-Daten
        analytics_df = pd.DataFrame(analytics_data)
        
        # Merge beide DataFrames
        final_df = pd.merge(video_df, analytics_df, on='video_id', how='left')
        
        # Speichere Ergebnis und stelle sicher, dass Videolänge als Ganzzahl gespeichtert wird
        video_df['video_length_seconds'] = video_df['video_length_seconds'].astype(int)
        output_filename = f'youtube_complete_analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        final_df.to_csv(output_filename, index=False)
        
        print(f"\nAnalyse abgeschlossen!")
        print(f"Daten wurden in {output_filename} gespeichert")
        print(f"Anzahl der Videos: {len(final_df)}")
        print(f"Anzahl der Spalten: {len(final_df.columns)}")
        
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {str(e)}")

if __name__ == "__main__":
    main()

Zusammenführen aller Monate 

In [ ]:
import os
import pandas as pd

# Liste für gesammelte DataFrames
all_data = []

# Dateien im aktuellen Arbeitsverzeichnis durchlaufen
for filename in os.listdir():
    # Prüfen, ob der Dateiname mit "youtube_complete_analysis" beginnt und auf ".csv" endet
    if filename.startswith("youtube_complete_analysis") and filename.endswith(".csv"):
        df = pd.read_csv(filename)
        all_data.append(df)

# Alle DataFrames zusammenfügen
combined_df = pd.concat(all_data, ignore_index=True)

# Zusammengeführte Datei speichern
output_filename = "youtube_complete_analysis_2024.csv"
combined_df.to_csv(output_filename, index=False)

print(f"Zusammenführung abgeschlossen! Datei gespeichert als: {output_filename}")
